In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    # si usan colab, deben cambiar el token de esta url
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/vHoneyNeonic_v02.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_08_Data_Wrangling/Data/vHoneyNeonic_v02.csv?token=AA4GFHLGET6MJARMYMYU2M26WM246
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

$\hspace{.5cm}$[Definiciones](#section_definiciones)

$\hspace{.5cm}$[Tipos de variables](#section_tipos_variables)

[Dataset](#section_dataset)

[Problema](#section_problema)

[Imports](#section_imports)

[Eliminación de duplicados](#section_eliminacion_duplicados)

[Reemplazo de valores](#section_reemplazo_valores)

[Renombre de índices](#section_renombre_indices)

[Discretización y binarización de variables](#section_discretizacion)

[Mapeo y transformación de datos](#section_map_transform)

[Variables categóricas y dummies](#section_cat_dummies)

[Algunos métodos de la clase string](#section_string)


---


<a id="section_intro"></a> 
## Intro

[volver a TOC](#section_toc)

<a id="section_definiciones"></a> 
### Definiciones

**Data Wrangling**:

Proceso de limpieza y unificación de conjuntos de datos desordenados y complejos para facilitar su acceso, exploración, análisis o modelización posterior.

**Data munging**:

Proceso de limpiar, preparar y validar los datos.

**Extract, Transform and Load (ETL)**:

Extraer, transformar y cargar los datos.

**Exploratory data analysis (EDA)**:

Resumir las principales características de un dataset, a menudo con métodos visuales, sin usar un modelo estadístico o de machine learning. 


<a id="section_tipos_variables"></a> 
### Tipos de variables

Las variables pueden ser caracterizadas como:

* **Cuantitativas**: 

$\hspace{.5cm}$ toman valores numéricos, como en el caso de del ingreso de una persona  o el precio de una casa.  

* **Cualitativas**: 

$\hspace{.5cm}$ toman valores en una de K diferentes clases o categorías.

$\hspace{.5cm}$ Variables cualitativas con dos posibles valores se denomina binaria o dicotómica.

$\hspace{.5cm}$ Tipos de variable cualitativa:

$\hspace{.5cm}$ **Nominal/Categórica**:

$\hspace{.5cm}$ toman valores entre categorías nombradas.

$\hspace{.5cm}$ Se suele asignar valores o rótulos numéricos a las variables categóricas:Estado civil, 0 si soltero y 1 si casado y 2 si divorciado. Los números utilizados para rotular son arbitrarios. En general, el software asume que los valores numéricos reflejan cantidades algebraicas y, por tanto, un orden.

$\hspace{.5cm}$ La principal medida de posición es la moda. 

$\hspace{.5cm}$ La mediana y la media no están definidas (y en general cualquier operación numérica tampoco).

$\hspace{.5cm}$ **Ordinal**:

$\hspace{.5cm}$ Es una variable categórica con un orden definido. 

Un ejemplo puede ser la variable rango_etario, que toma valores "niño", "adolescente", "adulto", "adulto mayor" donde está definido un orden "niño" < "adolescente" < "adulto" < "adulto mayor"


* **Dummy (variable indicadora)** 

$\hspace{.5cm}$ son variables cualitativas que toma valores 0 o 1 para indicar la ausencia o presencia de algún atributo o efecto categórico.

$\hspace{.5cm}$ Formalmente una variable dummy puede ser expresada mediante una función indicadora:

\begin{equation}
  D_i = \mathbb{I}_A(x_i) = \begin{cases} 
    1, & \text{si $x_i \in$ A}\\  
    0, & \text{si $x_i \notin $ A}.
  \end{cases}
\end{equation}

$\hspace{.5cm}$**¿Cuál es la relación entre variables categóricas y variables dummies?**

 Una variable categórica con N categorías puede ser expresada en términos de N−1 variables dummies (one-hot encoding).

 Resuelve el problema de interpretar las etiquetas numéricas como un intervalo.

 Si las categorías tienen muchos valores aumenta considerablemente la dimensionalidad de los datos.



---

<a id="section_dataset"></a> 
## Dataset

[volver a TOC](#section_toc)


https://www.kaggle.com/kevinzmith/honey-with-neonic-pesticide

Este dataset está inspirado en la producción de miel en EE. UU. Comprende el período 1998-2017. 

Incluye datos del Proyecto de Síntesis Nacional de Pesticidas de USGS, lo que permite evaluar las conexiones estadísticas entre la producción de miel y el uso de pesticidas neonicotinoides (neónicos).

Los datos muestran algunos datos interesantes:
* El uso neónico en EE. UU. comenzó alrededor de 2003, pero la disminución de la producción de miel comenzó en años anteriores.
* Algunos estados, como Kansas, han visto devastación de colonias de abejas a partir de 2003, cuando el uso neónico recién comenzaba.
* Otros estados, como Dakota del Norte, tienen un número aproximadamente estable de colonias de abejas, a pesar del mayor uso de neonics

Los campos de este dataset son:

* numcol: Número de colonias productoras de miel. Las colonias productoras de miel son el número máximo de colonias de las cuales se extrajo miel durante el año. Es posible tomar miel de colonias que no sobrevivieron durante todo el año.

* yieldpercol: Rendimiento de miel por colonia. La unidad es libras

* totalprod: TProducción total (numcol x yieldpercol). La unidad es libras

* stocks: Se refiere a las existencias en poder de los productores. La unidad es libras

* priceperlb: Se refiere al precio promedio por libra en función de las ventas ampliadas. La unidad es dólares.

* prodvalue: Valor de producción (totalprod x priceperlb). La unidad es dólares.

From USGS Data

* nCLOTHIANIDIN: La cantidad en kg de CLOTHIANIDIN aplicado

* nIMIDACLOPRID: La cantidad en kg de IMIDACLOPRID aplicado

* nTHIAMETHOXAM: La cantidad en kg de THIAMETHOXAM aplicado

* nACETAMIPRID: La cantidad en kg de ACETAMIPRID aplicado

* nTHIACLOPRID: La cantidad en kg de THIACLOPRID aplicado

* nAllNeonic: La cantidad en kg de todos los Neonics aplicados = (nCLOTHIANIDIN + nIMIDACLOPRID + nTHIAMETHOXAM + nACETAMIPRID + nTHIACLOPRID)



---

<a id="section_problema"></a> 
## Problema

[volver a TOC](#section_toc)


Usaremos los datos de producción de miel en EEUU para presentar y ejercitar métodos de pandas muy usados en las tareas de data wrangling.

---

<a id="section_imports"></a> 
## Imports

[volver a TOC](#section_toc)


In [ ]:
import pandas as pd
import numpy as np

Leemos los datos, vemos de qué tipos son las columnas, y mostramos los primeros registros del DataFrame para verificar que cargamos bien los datos.

In [ ]:
data_location = "../Data/vHoneyNeonic_v02.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Construimos un nuevo DataFrame que tiene la información asociada a los distintos estados de EEUU 

In [ ]:
data_state = data.loc[:, ["state", "StateName", "Region"]]

---

<a id="section_eliminacion_duplicados"></a> 

## Eliminación de duplicados

[volver a TOC](#section_toc)


Para determinar si un DataFrame tiene registros duplicados, primero debemos **definir en qué campos deben coincidir los valores de los registros para considerar que son iguales**.

El método `duplicated` de pandas permite definir en el argumento `subset` una lista de columnas a evaluar cuando queremos determinar si dos registros son duplicados. Si no establecemos el valor de este argumento, sólo seran duplicados aquellos registros cuyos valores coincidan en todos los campos.

El argumento `keep` toma valores
* first: devuelve False (no duplicado) el primer registro encontrado, y True (duplicados) todas las demás apariciones
* last: devuelve False (no duplicado) el último registro encontrado, y True (duplicados) todas las demás apariciones
* False: devuelve True (duplicado) todos los registros duplicados

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html


El método `drop_duplicates` de pandas devuelve un nuevo DataFrame sin registros duplicados.Los argumentos `subset` y `keep` tienen el mismo comportamiento que en `duplicated`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

**Ejercicio**:
    
A partir del DataFrame data, construyamos otro DataFrame con los datos de estado: "state", "StateName", "Region" y asignemoslo a la variable data_state.

Veamos si tiene registros duplicados de dos formas distintas:
* Evaluando data
* Evaluando data_state

Creemos otro DataFrame sin registros duplicados.

In [ ]:
data_state = data.loc[:, ["state", "StateName", "Region"]]

#Evaluando data_state
data_state_duplicates_mask = data_state.duplicated(keep="first")
print("registros duplicados en data_state: ", any(data_state_duplicates_mask))
print("cantidad de registros duplicados en data_state: ", data_state_duplicates_mask.sum())

#Evaluando data
data_duplicates_statefields =  data.duplicated(subset=["state", "StateName", "Region"], keep="first")
print("registros duplicados en data (considerando datos de estado): ", any(data_duplicates_statefields))
print("cantidad de registros duplicados en data (considerando datos de estado): ", data_duplicates_statefields.sum())

data_state_nodup = data_state.drop_duplicates(keep="first")
print("tam con duplicados: ", data_state.shape)
print("tam sin duplicados: ", data_state_nodup.shape)

Entonces, podríamos mantener los datos de estado en un DataFrame independiente, y mantener en cada registro un campo identificador de estado (que puede ser `state` en este ejemplo) que nos permita mapear con los datos adicionales.

Observemos que si bien contamos duplicados en data y en data_state, sólo hacemos drop en data_state. Si lo hicieramos en data perderíamos toda la información de produción por año para cada estado.

---

<a id="section_reemplazo_valores"></a> 

## Reemplazo de valores

[volver a TOC](#section_toc)


El método `replace` ofrece varias formas de efectuar reemplazos sobre una serie de Pandas:

* Un valor viejo por un valor nuevo.
    
* Una lista de valores viejos por un valor nuevo.
    
* Una lista de valores viejos por una lista de valores nuevos.
    
* Un diccionario que mapee valores nuevos y viejos.


https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html



### Un valor viejo por un valor nuevo

Reemplacemos las apariciones de "South" por "south"

In [ ]:
data_south = data.replace("South", "south")
data_south.head(3)

### Una lista de valores viejos por un valor nuevo

Reemplacemos todos los valores del campo totalprod por np.NaN

Para ese creemos una lista de valores únicos con los valores de la columna data.totalprod

In [ ]:
to_be_replaced = data.totalprod.unique()
data_totalprod_null = data.replace(to_be_replaced, np.NaN)

Observemos que en este reemplazo no estamos especificando sobre qué columna estamos reemplazando, entonces reemplaza los valores que están en el primer argumento en cualquier campo que aparezcan.

Veamos qué porcentaje de nulos había en el DatFrame original y cuántos hay en data_totalprod_null

In [ ]:
data.isnull().sum() / data.shape[0]

In [ ]:
data_totalprod_null.isnull().sum() / data_totalprod_null.shape[0]

Vemos que reemplazó todos los valores de `totalprod` y algunos de los valores en `numcol` `stocks` y `prodvalue`

### Una lista de valores viejos por una lista de valores nuevos

Reemplacemos los valores del campo state por otros que sean el resultado de concatenar el campo region con el campo state, para cada valor de state.

In [ ]:
valores_viejos = data.state.unique()
valores_nuevos = (data.Region + "_" + data.state).unique()
print(len(valores_viejos))
print(len(valores_nuevos))

In [ ]:
data_state_modif = data.replace(valores_viejos, valores_nuevos)
data_state_modif.sample(5)

Observemos que las listas de valores viejos y valores nuevos deben tener la misma longitud.

### Un diccionario que mapee valores nuevos y viejos

Hagamos el mismo reemplazo que en el punto anterior pero usando un diccionario.

En este caso se reemplazaran todas las apariciones en el DataFrame de las claves del diccionario por los valores asociados a esas claves.

In [ ]:
mapping = dict(zip(valores_viejos, valores_nuevos))
mapping

In [ ]:
data_dict = data.replace(mapping)
data_dict.sample(7)

---

<a id="section_renombre_indices"></a> 

## Renombre de índices 

[volver a TOC](#section_toc)


El método `rename` nos permite cambiar las etiquetas de los ejes índices y columnas.

El primer argumento `mapper` es un diccionario que tiene como claves las etiquetas a reemplazar y como valores las etiquetas de reemplazo.

El argumento `axis` indica si el reemplazo es sobre el índice o sobre las columnas

El argumento `index` es un diccionario. `mapper, axis=0` es equivalente a  `index=mapper`

El argumento `columns` es un diccionario. `mapper, axis=1` es equivalente a `columns=mapper`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html

Renombremos dos columnas usando los argumentos mapper y axis

In [ ]:
mapper = {
    'nIMIDACLOPRID': 'nimidacloprid', 
    'nTHIAMETHOXAM': 'nthiamethoxam'    
}
data_renamed_cols = data.rename(mapper, axis = 1)
print(data.columns)
print(data_renamed_cols.columns)

Repitamoslo usando el argumento columns

In [ ]:
data_renamed_cols_2 = data.rename(columns = mapper)
print(data.columns)
print(data_renamed_cols_2.columns)

Observemos que sólo se modifican las columnas (o índices) que están especificados en el diccionario de mapping. No hace falta incluir aquellos que no queremos renombrar

---

<a id="section_discretizacion"></a> 

## Discretización y binarización de variables

[volver a TOC](#section_toc)


El proceso de transformar una variable numérica en categórica se llama discretización. 

El método `cut` separa valores en intervalos discretos, devolviendo el intervalo semi-cerrado al que pertenece cada valor. Permite transformar variables continuas en categóricas .

El argumento `bins` puede ser un entero que indica la cantidad de intervalos de igual "ancho" a construir, o puede ser una lista en la que especificamos los límites de cada categoría.

El argumento `right` indica si el intervalos incluye el valor límite por la derecha.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html


Queremos definir categorias sobre la variable `numcol` que representa la cantidad de colonias productoras.

Veamos algunas estadísticas sobre esta variable


In [ ]:
data.numcol.describe()

Vemos que el mínimo es 2000 y el máximo 510.000.
Definamos estascategoriás:
* entre 2000 y 10000    
* entre 10000 y 300000
* entre 300000 y 65000
* más de 65000

Pra eso construimos la lista bins con esos valores

In [ ]:
# Defino los valores de corte
bins = [2000, 10000, 30000, 65000, 600000]

numcol_categories = pd.cut(data.numcol, bins, right=False)
numcol_categories.dtype

Y como esta variable es categórica, podemos usar el método `value_counts` para saber cuantos registros hay en cada categoría.

¿Qué pasa si usamos `value_counts` sobre `data.numcol`?

In [ ]:
numcol_categories.value_counts()

Creamos categorías con etiquetas asociadas usando el argumento `labels`

In [ ]:
group_labels = ['muy pocas', 'pocas', 'normal', 'muchas' ]
numcol_categories_labels = pd.cut(data.numcol, bins, labels=group_labels)
numcol_categories_labels.value_counts()

### Cuantilos

Una alternativa a especificar arbitrariamente los límites de los intervalos es usar cuantilos. 

Por definición en cada uno de esos conjuntos habrá igual cantidad de elementos.

pandas provee el método `qcut` para construirlos.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html

Usando qcut construyamos cuatro categorías para los valores de `data.numcol`

Estableciendo en True el valor del argumento `retbins` el método devuelve la categorías y los valores umbral de los bins construídos

In [ ]:
numcol_qcategories, numcol_qbins = pd.qcut(data.numcol, 4, labels = group_labels, retbins=True)
numcol_qcategories.value_counts()

In [ ]:
numcol_qbins

---

<a id="section_map_transform"></a> 

## Mapeo y transformación de datos

[volver a TOC](#section_toc)


A partir de un diccionario, podemos crear una nueva columna en un Dataframe donde 

1) las claves del diccionario se vinculen con la Series de valores de una columna del DataFrame original

2) y para cada registro se asigne, en la nueva columna, el valor del diccionario cuya clave es el valor de la Serie 1) en ese registro.

**Ejercicio**:

Usando un diccionario cambiemos los valores de las categorias definidas en el ejercicio anterior para numcol, para todos los registros de data.

Agregamos las categorías de numcol asociadas a cada registro al DatFrama data

In [ ]:
# renombro la series para que no queden dos columnas con el mismo nombre como resultado de concat:
numcol_qcategories.name = "numcol_cat"
data_numcol_cat = pd.concat([data, numcol_qcategories], axis = 1, )
print(data_numcol_cat.columns)
print(data.shape)
print(data_numcol_cat.shape)

Construyamos un diccionario que defina este mapeo y usemos ese diccionario para cambiar los valores originales
    
* pocas -> insuficiente
* muy pocas -> insuficiente
* normal -> suficiente
* muchas -> suficiente

Las claves están asociadas a la columna `numcol_cat` y los nuevos valores de cada registro estarán determinados por el valor de ese registro en la columna `numcol_cat` y el mapeo que define el diccionario.

In [ ]:
numcol_cat_mapper = {
   'pocas': 'insuficiente', 
    'muy pocas': 'insuficiente',
    'normal': 'suficiente',
    'muchas': 'suficiente'
}

In [ ]:
data_numcol_cat['numcol_cat'] = data_numcol_cat['numcol_cat'].map(numcol_cat_mapper)
data_numcol_cat.sample(5)

Observemos que aplicamos map sobre una columna, no sobre el DataFrame compelto como hacemos con el método `replace`

In [ ]:
data_numcol_cat.numcol_cat.value_counts()

---

<a id="section_cat_dummies"></a> 

## Variables categóricas y dummies

[volver a TOC](#section_toc)

Pandas provee con el método pd.get_dummies que recibe una Serie o una lista de Series y realiza el one hot encoding.

Recordemos que una variable con k categorías se puede representar con k-1 variables.

Por eso un parámetro clave de pd.get_dummies es drop_first = True que genera k-1 categorías en lugar de k.

El argumento prefix nos permite establecer un prefijo para el nombre de cada una de las columnas de categorías que representan la variable. Este argumento es especialmente útil cuando construimos dummies de más de una columna de un DataFrame, para evitar confusiones sobre cuál es las variable original que corresponde a cada columna de categoría.

**Ejercicio**:

Creemos variabels dummies que representen los valores de numcol_qcategories

In [ ]:
numcol_qcategories_dummies = pd.get_dummies(numcol_qcategories, drop_first = True, prefix='numcol')
numcol_qcategories_dummies

Observemos que eliminó la primer categoría "muy pocas"

---

<a id="section_string"></a> 

## Algunos métodos de la clase string

[volver a TOC](#section_toc)


https://docs.python.org/3/library/string.html

A continuación presentamos algunos métodos sobre string que son muy usados en data wrangling, aplicados en funciones lambda.


### `split`

Devuelve una lista de palabras en un string separadas por el delimitador pasado como parámetro

https://docs.python.org/3/library/stdtypes.html#str.split

In [ ]:
cadena = 'a,b,  guido, asjd, kle, askl'
separador = ','
cadena_en_partes = cadena.split(separador)
cadena_en_partes 

### `strip`

Devuelve una copia de un string, eliminando del comienzo y fin de la cadena los caracteres pasados como parámetro. Si no especificamos el valor del argumento, elimina espacios.

https://docs.python.org/3/library/stdtypes.html#str.strip

In [ ]:
texto = "   Este es el primer ejemplo....wow!!!   ";
texto_sin_espacios = texto.strip()
print(texto_sin_espacios)

texto1 = "0000000Este es el segundo ejemplo....wow!!!0000000";
texto1_sin_ceros = texto1.strip('0')
print(texto1_sin_ceros)

texto2 = "    0000000Este es el segundo ejemplo....wow!!!0000000";
texto2_sin_ceros = texto2.strip('0')
print(texto2_sin_ceros)


Observemos que en texto2 no eliminó los espacios iniciales, sólo el caracter establecido en el argumento.

### `find`

Devuelve el mínimo índice donde encontramos como substring el valor pasado como parámetro.

Si no lo encuentra devuelve -1

https://docs.python.org/3/library/stdtypes.html#str.find

In [ ]:
cadena

In [ ]:
cadena.find(':')

In [ ]:
cadena.find('as')

In [ ]:
cadena.find('asj')

In [ ]:
cadena.find('asp')

Observemos que devuelve el índice del comienzo del string buscado

### `index`

Es similar a find pero devuelve una excepción de tip `ValueError` cuando no encuentra el valor buscado

https://docs.python.org/3/library/stdtypes.html#str.index

In [ ]:
cadena.index(',')

In [ ]:
cadena.index(':')

### `count`

Devuelve la cantidad de apariciones del valor pasado como parámetro

https://docs.python.org/3/library/stdtypes.html#str.count

In [ ]:
cadena.count(',')

In [ ]:
cadena.count('as')

### `replace`

Devuelve una copia del string con todas las ocurrencias del primer argumento reemplazadas por el segundo argumento

https://docs.python.org/3/library/stdtypes.html#str.replace

In [ ]:
cadena_punto_y_coma = cadena.replace(',', ';')
cadena_punto_y_coma

In [ ]:
cadena_qw = cadena.replace('as', 'qw')
cadena_qw

---

#### Referencias

Python for Data Analysis. Wes McKinney. Cap 12


https://docs.python.org/3/library/string.html